<a href="https://colab.research.google.com/github/thegreatgupta/ObjectRecognition/blob/master/MobileNet_TensorFlow_with_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Confirm TensorFlow can see the GPU

Simply select "GPU" in the Accelerator drop-down in Notebook Settings (either through the Edit menu or the command palette at cmd/ctrl-shift-P).

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [3]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# Runs the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
10.725042377000136
GPU (s):
0.18642497699966043
GPU speedup over CPU: 57x


In [0]:
import tarfile

In [0]:
tar = tarfile.open("object-classification-dataset.tar")

In [0]:
tar.extractall()

In [0]:
tar.close()

In [0]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.applications.xception import Xception
from keras.applications.resnet50 import ResNet50
from keras.applications.mobilenet import MobileNet
from keras.layers import Dense,GlobalAveragePooling2D
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

In [0]:
def model_Xception(img_shape=(299, 299, 3), n_classes=50,
                   load_pretrained=False, freeze_layers_from='base_model'):
    # Decide if load pretrained weights from imagenet
    if load_pretrained:
        weights = 'imagenet'
    else:
        weights = None

    # Get base model
    #base_model = Xception(include_top=False, weights=weights,
                       #input_tensor=None, input_shape=img_shape)
    
    #base_model = ResNet50(include_top=False, weights=weights,
                       #input_tensor=None, input_shape=img_shape)
    
    base_model = MobileNet(include_top=False, weights=weights,
                       input_tensor=None, input_shape=img_shape)

    # Add final layers
    x = base_model.output
    x = Dense(n_classes, activation='relu')(x)
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(n_classes, activation='softmax')(x)

    # This is the model we will train
    model = Model(input=base_model.input, output=predictions)
    
    # Model Summary
    # print(model.summary())
    
    # # let's visualize layer names and layer indices to see how many layers/blocks to re-train
    # # uncomment when choosing based_model_last_block_layer
    # for i, layer in enumerate(model.layers):
    #     print(i, layer.name)

    # Freeze some layers
    if freeze_layers_from is not None:
        if freeze_layers_from == 'base_model':
            print ('   Freezing base model layers')
            for layer in base_model.layers:
                layer.trainable = False
        else:
            for i, layer in enumerate(model.layers):
                print(i, layer.name)
            print ('   Freezing from layer 0 to ' + str(freeze_layers_from))
            for layer in model.layers[:freeze_layers_from]:
               layer.trainable = False
            for layer in model.layers[freeze_layers_from:]:
               layer.trainable = True

    print(model.summary())
    return model

In [0]:
def train_Xception(train_set_path, val_set_path, validation_split = 0.2,
                   batch_size = 32, class_mode = 'categorical', horizontal_flip = False,
                   vertical_flip = False, rotation_range = None, target_size = (299, 299),
                   model = None, epochs = 1):
    
    # Load Image Data Set Using Keras
    train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = horizontal_flip,
                                   vertical_flip = vertical_flip,
                                   rotation_range=rotation_range,
                                   validation_split=validation_split)
    
    training_set = train_datagen.flow_from_directory(train_set_path,
                                                 target_size = target_size,
                                                 batch_size = batch_size,
                                                 class_mode = class_mode,
                                                 subset = 'training')
    
    validation_set = train_datagen.flow_from_directory(val_set_path,
                                                 target_size = target_size,
                                                 batch_size = batch_size,
                                                 class_mode = class_mode,
                                                 subset = 'validation')
    
    
    # Compile Model
    learning_rate = 0.0001
    loss = 'categorical_crossentropy'
    # opt_rms = keras.optimizers.rmsprop(lr=learning_rate,decay=1e-6)
    adam_optimizer = Adam(lr = learning_rate)
    model.compile(optimizer = adam_optimizer, loss = loss, metrics = ['accuracy'])
    
    model.fit_generator(training_set,
                        steps_per_epoch = training_set.samples,
                        epochs = epochs,
                        validation_data = validation_set,
                        validation_steps = validation_set.samples)
    
    
    return model

In [0]:
# Declare Constants
img_shape = (128, 128, 3)
target_size = (128, 128)
n_classes = 49
epochs = 10
train_set_path = '101_ObjectCategories' #'dataset2/training_set_test'
val_set_path = train_set_path
validation_split = 0.15
batch_size = 16
horizontal_flip = True
rotation_range = None
learning_rate = 0.0001
loss = 'categorical_crossentropy'

In [45]:
# Build Model
model = model_Xception(img_shape = img_shape,  n_classes = n_classes, load_pretrained = True)

17227776/17225924 [==============================] - 2s 0us/step
   Freezing base model layers
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 129, 129, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
_______________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [46]:
model = train_Xception(train_set_path = train_set_path, val_set_path = val_set_path, validation_split = validation_split,
                       batch_size = batch_size, horizontal_flip = horizontal_flip, rotation_range = rotation_range,
                       model = model, epochs = epochs, target_size = target_size)

Found 4584 images belonging to 49 classes.
Found 782 images belonging to 49 classes.
Epoch 1/10
4584/4584 [==============================] - 577s 126ms/step - loss: 0.7539 - acc: 0.8199 - val_loss: 0.3317 - val_acc: 0.8983
Epoch 2/10
4584/4584 [==============================] - 579s 126ms/step - loss: 0.1453 - acc: 0.9640 - val_loss: 0.3118 - val_acc: 0.9010
Epoch 3/10
4584/4584 [==============================] - 580s 127ms/step - loss: 0.0815 - acc: 0.9799 - val_loss: 0.2787 - val_acc: 0.9092
Epoch 4/10
4584/4584 [==============================] - 577s 126ms/step - loss: 0.0574 - acc: 0.9856 - val_loss: 0.2587 - val_acc: 0.9191
Epoch 5/10
4584/4584 [==============================] - 582s 127ms/step - loss: 0.0442 - acc: 0.9891 - val_loss: 0.2446 - val_acc: 0.9248
Epoch 6/10
4584/4584 [==============================] - 578s 126ms/step - loss: 0.0360 - acc: 0.9907 - val_loss: 0.2824 - val_acc: 0.9129
Epoch 7/10
4584/4584 [==============================] - 572s 125ms/step - loss: 0.0298 

In [0]:
model.save_weights('obj_recognition_xception_normal_bt16_ep10.h5')